# Implementation Notes

The internals of ChebPy have been designed to resemble the design structure of MATLAB Chebfun. The Chebfun v5 class diagram thus provides a useful map for understanding how the various pieces of ChebPy fit together (diagram courtesy of the Chebfun team, available [here](https://github.com/chebfun/chebfun/wiki/Class-diagram)):

![](./chebfun-v5-class-diag.png)


At this stage, only a relatively small subset of MATLAB Chebfun has been implemented in ChebPy. In the class diagram above, this consists of all the classes traced by the path going up from from `Chebtech2` (green box in the bottom right), to `Chebfun` (blue box near the top-left). More explicitly, the following classes currently exist in ChebPy:

- `Chebfun` ([core/chebfun.py](https://github.com/chebpy/chebpy/blob/master/chebpy/core/chebfun.py))
- `Fun` ([core/fun.py](https://github.com/chebpy/chebpy/blob/master/chebpy/core/fun.py))
- `Classicfun` ([core/classicfun.py](https://github.com/chebpy/chebpy/blob/master/chebpy/core/classicfun.py))
- `Bndfun` ([core/bndfun.py](https://github.com/chebpy/chebpy/blob/master/chebpy/core/bndfun.py))
- `Onefun` ([core/onefun.py](https://github.com/chebpy/chebpy/blob/master/chebpy/core/onefun.py))
- `Smoothfun` ([core/smoothfun.py](https://github.com/chebpy/chebpy/blob/master/chebpy/core/smoothfun.py))
- `Chebtech` ([core/chebtech.py](https://github.com/chebpy/chebpy/blob/master/chebpy/core/chebtech.py))
- `Chebtech2` ([core/chebtech.py](https://github.com/chebpy/chebpy/blob/master/chebpy/core/chebtech.py))

ChebPy additionally provides the following classes which do not appear in their present form in MATLAB Chebfun:

- `Interval` ([core/utilities.py](https://github.com/chebpy/chebpy/blob/master/chebpy/core/utilities.py))
- `Domain` ([core/utilities.py](https://github.com/chebpy/chebpy/blob/master/chebpy/core/utilities.py))

ChebPy allows the user to override some default preferences, similar to what is available through `chebfunpref` in `Chebfun` (bottom right). Not all options are the same, and chebpy allows additional customisation not found in `Chebfun`.

- `UserPrefs` ([core/settings.py](https://github.com/chebpy/chebpy/blob/master/chebpy/core/settings.py))

The general rule is that each ChebPy class lives in its own python file.

## Demo
One can explore the organisation of the library in practice as follows. We describe the three core components with reference to the colours in the above class diagram.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

### Chebfun (blue)

We'll start by creating an identity chebfun on an arbitrary non-unit interval:

In [ ]:
from chebpy import chebfun

x = chebfun('x', [-2, 3])
x

This variable is an object of class `Chebfun`:

In [ ]:
type(x)

### Fun (yellow)

Conceptually, `Chebfun` objects are defined as a collection (numpy array) of Fun objects. One can access these via the `.funs` attribute, and in this example, since our function is globally smooth, our chebfun is composed of single Fun:

In [ ]:
x.funs

Fun is an abstract class, so we don't actually see Fun objects per-se, but rather objects further down the inheritance tree. Specifically, we see objects of type `BndFun`, denoting a function defined on a bounded interval.

Here's a more interesting function, this time composed of two `Fun` objects and created by inducing a discontinuity via use of the maximum operator:

In [ ]:
f = np.sin(3*x).maximum(-np.sin(x))
f

In [ ]:
f.funs

The core `Chebfun` class constructor accepts iterable collections of `Fun` objects, and the above chebfun can be recreated as follows:

In [ ]:
from chebpy.core.chebfun import Chebfun
Chebfun(f.funs)

The Fun objects defining a chebfun must provide a non-overlapping and complete partition of the global approximation interval. ChebPy-specific exceptions will be raised if the user/developer does not properly account for this.

To see this, let's break the second Fun into two sub-pieces (using `fun.restrict`) and attempt various reconstruction permutations.

In [ ]:
from chebpy.core.utilities import Interval

_, a, c = f.breakpoints
b = 1
fun0 = f.funs[0]
fun1 = f.funs[1].restrict(Interval(a,b))
fun2 = f.funs[1].restrict(Interval(b,c))
fun0.plot(linewidth=3); fun1.plot(linewidth=3); fun2.plot(linewidth=3);
plt.show()

So, the following works:

In [ ]:
Chebfun([fun0,fun1,fun2])

But on the other other hand, the following raises an exception:

In [ ]:
Chebfun([fun0,fun2])

So also does this:

In [ ]:
Chebfun([fun0,f.funs[1],fun1])

### Onefun (green)

A Fun object is defined by the composition of `Onefun` object and an Interval object. A `Onefun` object in ChebPy, as with MATLAB Chebfun, define a set of core approximation behaviour on the unit interval [-1,1]. The computational mechanics of mapping these operations to arbitrary intervals [a,b] is managed, in part, by the a corresponding Interval object.

To illustrate, let's take the first component `Fun` from earlier (which was specifically a `Bndfun`):

In [ ]:
f.funs[0]

The Onefun and Interval object components are accessed as follows:

In [ ]:
print(f.funs[0].onefun)
print(f.funs[0].interval)


`Onefun` is an abstract class, so what we actually see instantiated is an object of type `Chebtech2`. To see that the `Onefun` object is what is claims to be (a representation defined on the unit-interval), we can plot it (users will rarely do this in practice, but this can nevertheless be a useful feature for developers):

In [ ]:
f.funs[0].onefun.plot(linewidth=3)
plt.show()

So while the `Onefun` attribute determines approximation behaviour, the interval attribute manages the mapping to and from the approximation interval to [-1,1]. So for instance, one can evaluate the interval object at some set of points in [-1,1] and obtain these values mapped to [a,b]:

In [ ]:
f.funs[0].interval(np.linspace(-1,1,11))

## User preferences (`UserPrefs`)

The user may want to specify different tolerances, for example if speed is important or the function under consideration is particularly difficult. It is also possible to change default behaviour like plotting.

In [ ]:
import chebpy  # import entire module to access lower-level parts

chebpy.core.settings.userPrefs.eps = 1e-10  # lower the tolerance in chebpy
cheb = chebpy.chebfun(lambda x: x**2)
chebpy.core.settings.userPrefs.N_plot = 21  # use fewer points for plotting
cheb.plot(marker='x', label='N_plot = 21')
chebpy.core.settings.userPrefs.reset('N_plot')  # restore default
cheb.plot(label='default N_plot')
plt.legend()
plt.show()